In [1]:
import pyforest
import requests
from bs4 import BeautifulSoup

pyforest.lazy_imports()

['import skimage',
 'import imutils',
 'from sklearn import svm',
 'import xgboost as xgb',
 'from sklearn.model_selection import KFold',
 'import fastai',
 'import os',
 'from sklearn.linear_model import ElasticNetCV',
 'import pickle',
 'from sklearn.preprocessing import OneHotEncoder',
 'from sklearn.manifold import TSNE',
 'import awswrangler as wr',
 'from sklearn.linear_model import ElasticNet',
 'import sys',
 'import bokeh',
 'from sklearn.preprocessing import MinMaxScaler',
 'import dash',
 'import cv2',
 'from sklearn.impute import SimpleImputer',
 'from pyspark import SparkContext',
 'from openpyxl import load_workbook',
 'from sklearn.preprocessing import LabelEncoder',
 'import spacy',
 'import matplotlib.pyplot as plt',
 'import torch',
 'from sklearn.model_selection import GridSearchCV',
 'from sklearn.model_selection import cross_val_score',
 'from sklearn.feature_extraction.text import TfidfVectorizer',
 'from scipy import signal as sg',
 'from sklearn.ensemble import 

In [2]:
response = requests.get("https://www.imdb.com/search/title?genres=sci-fi&")

In [3]:
response.headers

{'Content-Type': 'text/html;charset=UTF-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'Server', 'Date': 'Fri, 14 Oct 2022 12:47:54 GMT', 'x-amz-rid': 'J314XQXPDC2EXZKZ83BQ', 'Set-Cookie': 'uu=eyJpZCI6InV1YTNkMjNhOTJiNzc4NGYzNmE1ZGUiLCJwcmVmZXJlbmNlcyI6eyJmaW5kX2luY2x1ZGVfYWR1bHQiOmZhbHNlfX0=; Domain=.imdb.com; Expires=Wed, 01-Nov-2090 16:02:01 GMT; Path=/; Secure, session-id=000-0000000-0000000; Domain=.imdb.com; Expires=Wed, 01-Nov-2090 16:02:01 GMT; Path=/; Secure, session-id-time=2296471673; Domain=.imdb.com; Expires=Wed, 01-Nov-2090 16:02:01 GMT; Path=/; Secure', 'X-Frame-Options': 'SAMEORIGIN', 'Content-Security-Policy': "frame-ancestors 'self' imdb.com *.imdb.com *.media-imdb.com withoutabox.com *.withoutabox.com amazon.com *.amazon.com amazon.co.uk *.amazon.co.uk amazon.de *.amazon.de translate.google.com images.google.com www.google.com www.google.co.uk search.aol.com bing.com www.bing.com", 'Content-Language': 'en-US', 'Strict-Transport-Security': '

In [35]:
content = response.text
parser = BeautifulSoup(content, 'html.parser')
test = parser.find('div', class_ = 'lister-item-content')
test


<!DOCTYPE html>

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<script type="text/javascript">var IMDbTimer={starttime: new Date().getTime(),pt:'java'};</script>
<script>
    if (typeof uet == 'function') {
      uet("bb", "LoadTitle", {wb: 1});
    }
</script>
<script>(function(t){ (t.events = t.events || {})["csm_head_pre_title"] = new Date().getTime(); })(IMDbTimer);</script>
<title>Top 50 Sci-Fi Movies and TV Shows - IMDb</title>
<script>(function(t){ (t.events = t.events || {})["csm_head_post_title"] = new Date().getTime(); })(IMDbTimer);</script>
<script>
    if (typeof uet == 'function') {
      uet("be", "LoadTitle", {wb: 1});
    }
</script>
<script>
    if (typeof uex == 'function') {
      uex("ld", "LoadTitle", {wb: 1});
    }
</script>
<link href="https://www.imdb.com/search/title/?genres=sci-fi" rel="canonical"/>
<meta content="http://www.imdb.com/search/title/?genres=sci-fi" property="og:url">
<script>
  

In [5]:
test_name = test.h3.a.text
test_name

'Andor'

In [6]:
test_rating_meta = test.find('div', class_ = 'inline-block ratings-metascore')
if test_rating_meta is not None:
    test_rating_meta.span.text
else:
    print("Não tem")

Não tem


In [7]:
movie_data = parser.find_all('div', class_ = 'lister-item-content')
len(movie_data)

50

In [44]:
movie_names = []
imdb_scores = []
metascores = []
gross_earned = []
votes = []
genres = []

for movie in movie_data:
    name = movie.h3.a.text
    ratings_imdb = movie.find('div', class_ = 'inline-block ratings-imdb-rating')
    try:
        imdb = float(ratings_imdb.strong.text)
    except:
        imdb = None
    ratings_meta = movie.find('div', class_ = 'inline-block ratings-metascore')
    g = movie.find_all('span', attrs = {'name':'nv'})
    try:
        user_votes = g[0].text
    except:
        user_votes = None
    
    movie_genre = movie.find('span', class_ = 'genre').text.strip()
    
    if len(g) == 2:
        gross = g[1].text
    else:
        gross = '$0M'
        
    if ratings_meta is None:
        metascore = 0
    else:
        metascore = int(ratings_meta.span.text.strip())
        
    movie_names.append(name)
    imdb_scores.append(imdb)
    metascores.append(metascore)
    gross_earned.append(gross)
    votes.append(user_votes)
    genres.append(movie_genre)

In [45]:
print(len(movie_names), len(imdb_scores), len(metascores), len(gross_earned), len(votes), len(genres), sep="\n")

50
50
50
50
50
50


In [46]:
movies_df = pd.DataFrame({'movie_name': movie_names, 'genres': genres, 'imdb_score': imdb_scores, 'metascore': metascores, 'user_votes': votes, 'gross_earned': gross_earned})
movies_df

<IPython.core.display.Javascript object>

,movie_name,genres,imdb_score,metascore,user_votes,gross_earned
0,Andor,"Action, Adventure, Drama",8.1,0,"[[112,975]]",$0M
1,Mulher-Hulk: Defensora de Heróis,"Action, Adventure, Comedy",5.1,0,"[[112,975]]",$0M
2,Super Mario Bros. O Filme,"Animation, Adventure, Comedy",NaN,0,"[[112,975]]",$0M
3,Pantera Negra: Wakanda para Sempre,"Action, Adventure, Drama",NaN,0,"[[112,975]]",$0M
4,The Munsters,"Comedy, Family, Fantasy",4.6,57,"[[112,975]]",$0M
5,História de Horror Americana,"Drama, Horror, Sci-Fi",8.0,0,"[[112,975]]",$0M
6,Rick e Morty,"Animation, Adventure, Comedy",9.2,0,"[[112,975]]",$0M
7,O Conto da Aia,"Drama, Sci-Fi, Thriller",8.4,0,"[[112,975]]",$0M
8,Stranger Things,"Drama, Fantasy, Horror",8.7,0,"[[112,975]]",$0M
9,Vesper,"Adventure, Drama, Sci-Fi",6.1,70,"[[112,975]]",$0M
